In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
!pip install transformers sentence-transformers faiss-cpu

In [1]:
# ✅ STEP 1: Fix library versions for Colab (LoRA + 4-bit quantization)
!pip install --upgrade --quiet bitsandbytes==0.43.1
!pip install --upgrade --quiet transformers==4.41.2 accelerate==0.31.0 peft==0.11.1

import os

In [2]:
!pip install langdetect deep-translator

from langdetect import detect
from deep_translator import GoogleTranslator
# Begin by running these  .....

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
from datasets import load_dataset
# relevant libraries

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
# loading the LLM and Embedding Model
# LLM
# much more stronger then last model(flat T-5 small)
id_model = "openchat/openchat-3.5-1210"

#loads the language model
tokenizer = AutoTokenizer.from_pretrained(id_model)

# notes: it loads ,tokenizer that turns text into numbers the model can understand.
print("model is loaded")

embedder = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model is loaded


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from google.colab import files
files.upload()
# uploaded this kaggle data set (livetsock diseases)

Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"sobaina","key":"fda3d4284a378a0a5daa7dae2a2fbc9d"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d researcher1548/livestock-symptoms-and-diseases


Dataset URL: https://www.kaggle.com/datasets/researcher1548/livestock-symptoms-and-diseases
License(s): unknown
livestock-symptoms-and-diseases.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
#  Unzip it
!unzip livestock-symptoms-and-diseases.zip

Archive:  livestock-symptoms-and-diseases.zip
replace animal_disease_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: animal_disease_dataset.csv  


In [10]:

import pandas as pd

df = pd.read_csv("animal_disease_dataset.csv")
df.head(2)


,Animal,Age,Temperature,Symptom 1,Symptom 2,Symptom 3,Disease
0,cow,3,103.1,depression,painless lumps,loss of appetite,pneumonia
1,buffalo,13,104.5,painless lumps,loss of appetite,depression,lumpy virus


In [11]:

  import json
# just converting the data from csv to json
  def csv_into_json(csv_file,file_jsonl):
    df = pd.read_csv("animal_disease_dataset.csv")
    with open(file_jsonl,"w",encoding='utf-8') as file :
       for _, row in df.iterrows():
            symptoms = ', '.join([str(row[col]) for col in ['Symptom 1', 'Symptom 2', 'Symptom 3'] if pd.notna(row[col])])
            instruction = (
                f"Animal: {row['Animal']}, Age: {row['Age']}, "
                f"Temperature: {row['Temperature']}°F, Symptoms: {symptoms}"
            )
            response = str(row['Disease'])
            example = {"instruction": instruction, "based on the knowledge i have it can be ": response}
            file.write(json.dumps(example, ensure_ascii=False) + '\n')




  csv_into_json('animal_disease_dataset.csv', 'livestock_finetune.jsonl')




In [12]:
!head livestock_finetune.jsonl
# file got converted

{"instruction": "Animal: cow, Age: 3, Temperature: 103.1°F, Symptoms: depression, painless lumps, loss of appetite", "based on the knowledge i have it can be ": "pneumonia"}
{"instruction": "Animal: buffalo, Age: 13, Temperature: 104.5°F, Symptoms: painless lumps, loss of appetite, depression", "based on the knowledge i have it can be ": "lumpy virus"}
{"instruction": "Animal: sheep, Age: 1, Temperature: 100.5°F, Symptoms: depression, painless lumps, loss of appetite", "based on the knowledge i have it can be ": "lumpy virus"}
{"instruction": "Animal: cow, Age: 14, Temperature: 100.3°F, Symptoms: loss of appetite, swelling in limb, crackling sound", "based on the knowledge i have it can be ": "blackleg"}
{"instruction": "Animal: sheep, Age: 2, Temperature: 103.6°F, Symptoms: painless lumps, loss of appetite, depression", "based on the knowledge i have it can be ": "pneumonia"}
{"instruction": "Animal: goat, Age: 10, Temperature: 101.2°F, Symptoms: loss of appetite, blisters on gums, di

In [ ]:
# using peft for 	Efficient fine-tuning (e.g., LoRA) and accelerate	 to Run training on any hardware setup
!pip install transformers datasets peft accelerate


In [13]:
!ls -lh livestock_finetune.jsonl

-rw-r--r-- 1 root root 7.5M Jul 15 13:19 livestock_finetune.jsonl


In [14]:
from datasets import Dataset
# Read file into memory as list of dicts
with open("livestock_finetune.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Create Hugging Face Dataset from in-memory list
dataset = Dataset.from_list(data)


In [15]:

# !pip install -U bitsandbytes
# import bitsandbytes as bnb
# print("bitsandbytes version:", bnb.__version__)

!pip install -U transformers


In [18]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
!pip install -U bitsandbytes accelerate peft

# !pip install -U bitsandbytes==0.41.0 accelerate peft


# Set environment variables for bitsandbytes and triton
import os
os.environ['FORCE_BITSANDBYTES'] = '1'
# os.environ['BITSANDBYTES_NOWARN'] = '1'


import bitsandbytes as bnb

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    id_model,
    quantization_config=bnb_config,
    device_map="auto"
)

  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached accelerate-1.8.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.16.0-py3-none-any.whl.metadata (14 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
Using cached accelerate-1.8.1-py3-none-any.whl (365 kB)
Using cached peft-0.16.0-py3-none-any.whl (472 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.31.0
    Uninstalling accelerate-0.31.0:
      Successfully uninstalled accelerate-0.31.0
  Attempting uninstall: peft
    Found existing installation: peft 0.11.1
    Uninstalling peft-0.11.1:
      Successfully uninstalled peft-0.11.1


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [23]:
!pip uninstall -y peft
!pip install --no-cache-dir peft==0.11.1

# Force Python to reload the new peft
import importlib, sys
if "peft" in sys.modules:
    importlib.reload(sys.modules["peft"])

from peft import get_peft_model, LoraConfig, TaskType

Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 89.5 MB/s eta 0:00:00


 LoRA and Prep for Training

In [24]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    # target_modules=["q_proj", "v_proj"],  # specific to Mistral/OpenChat
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 7,245,156,352 || trainable%: 0.0470


In [30]:
tokenizer = AutoTokenizer.from_pretrained(id_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # prevent padding errors

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
def format_and_tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['based on the knowledge i have it can be '] }"
    tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=778,  # fits data
        padding="max_length",
        return_tensors="pt"
    )

    tokenized = {k: v.squeeze(0) for k, v in tokenized.items()}
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized


tokenized_dataset = dataset.map(
    format_and_tokenize,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/43778 [00:00<?, ? examples/s]

In [34]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./lora-openchat",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,  # Changed from fp16=True
    tf32=True,  # Added tf32=True
    logging_steps=10,
    save_steps=200,
    save_total_limit=1,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:477: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  if self.has_fp8_handler:


In [35]:
# HERREEEEE COVNO MEMEORY STARTS
# --- Conversation memory ---
conversation_history = ""

def chat_with_model(user_message):
    """
    Handles ONE chat turn in English while keeping memory.
    """

    global conversation_history

    # Add user message to memory
    conversation_history += f"\nUser: {user_message}"

    # Build the prompt with history
    prompt = f"""You are a helpful veterinary assistant.
Remember the conversation context.

Conversation so far:
{conversation_history}

Assistant:"""

    # Send to model
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the new reply
    if "Assistant:" in response:
        response_cleaned = response.split("Assistant:")[-1].strip()
    else:
        response_cleaned = response.strip()

    # Save assistant reply in history
    conversation_history += f"\nAssistant: {response_cleaned}"

    return response_cleaned


In [37]:
# --- Language Detection + Translation ---
def detect_lang(text):
    """Detects language code like 'en', 'es', 'fr'."""
    try:
        return detect(text)
    except:
        return "en"  # fallback

def translate_to_en(text):
    """Translates any text into English."""
    return GoogleTranslator(source='auto', target='en').translate(text)

def translate_back(text, target_lang):
    """Translates English back to target language."""
    return GoogleTranslator(source='auto', target=target_lang).translate(text)


In [38]:
def multilingual_chat(user_message):
    """
    Handles multilingual chat:
    - Detects language
    - Translates to English if needed
    - Gets model response (with memory)
    - Translates back to original language
    """

    # Detect user language
    user_lang = detect_lang(user_message)

    # Translate to English if needed
    if user_lang != "en":
        translated_input = translate_to_en(user_message)
    else:
        translated_input = user_message

    # Get model's English reply
    response_en = chat_with_model(translated_input)

    # Translate back if needed
    if user_lang != "en":
        response_final = translate_back(response_en, user_lang)
    else:
        response_final = response_en

    return response_final


In [40]:
    print(multilingual_chat("میری گائے کو بخار ہے اور وہ کھا نہیں رہی"))
# بھینس دا وزن گھٹ رہا اے، خوراک وچ کی شامل کراں؟



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


مجھے یہ سن کر افسوس ہوا کہ آپ کی گائے ٹھیک نہیں ہو رہی ہے۔ بخار اور بھوک کا نقصان مویشیوں میں بیماری کی علامت ہوسکتا ہے۔ یہاں کچھ اقدامات ہیں جو آپ اپنی گائے کی مدد کے ل take لے سکتے ہیں:

1. ایک ویٹرنریرین سے رابطہ کریں: یہ ضروری ہے کہ کسی پیشہ ور افراد کا آپ کی گائے کی حالت کا اندازہ لگائیں اور مناسب علاج کی سفارش کریں۔

2. اپنے گائے کے درجہ حرارت کی نگرانی کریں: اپنی گائے کے درجہ حرارت پر نظر رکھیں اور کسی بھی تبدیلی کا نوٹ کریں۔ ایک ملاشی تھرمامیٹر آپ کی گائے کا درجہ حرارت لینے کا سب سے درست طریقہ ہے۔

3. تازہ پانی فراہم کریں: یقینی بنائیں کہ آپ کی گائے کو ہر وقت صاف ، تازہ پانی تک رسائی حاصل ہے۔ پانی کی کمی بیماری کی علامات کو بڑھا سکتی ہے۔

4. متوازن غذا پیش کریں: اگر آپ کا گائے نہیں کھا رہا ہے تو ، اسے متوازن غذا استعمال کرنے کی ترغیب دینے کی کوشش کریں۔ آپ اس کے پسندیدہ کھانے کی پیش کش کرنے یا غذائیت سے بھرپور ضمیمہ فراہم کرنے کی کوشش کر سکتے ہیں۔

5. رکھیں


In [42]:
 print(multilingual_chat("بھینس دا وزن گھٹ رہا اے، خوراک وچ کی شامل کراں؟"))


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


اگر آپ کے بھینس کا وزن کم ہے تو ، اس کی وجہ متعدد عوامل جیسے خراب غذائیت ، بیماری ، یا تناؤ کی وجہ سے ہوسکتا ہے۔ اپنے بھینس کے وزن کو بہتر بنانے میں مدد کے ل its ، اس کی غذا میں درج ذیل سپلیمنٹس کو شامل کرنے پر غور کریں:

1. پروٹین: ایک اعلی معیار کے پروٹین کا ماخذ شامل کرنا جیسے سویا بین کا کھانا ، روئی کا کھانا ، یا مچھلی کا کھانا آپ کے بھینسوں کو پٹھوں کے بڑے پیمانے پر تعمیر اور برقرار رکھنے میں مدد کرسکتا ہے۔

2. وٹامن اور معدنیات: آپ کے بھینس کے لئے متوازن غذا میں ضروری وٹامن اور معدنیات جیسے وٹامن اے ، ڈی ، ای ، اور کے ، نیز کیلشیم ، فاسفورس اور میگنیشیم شامل ہونا چاہئے۔ یہ غذائی اجزاء مجموعی صحت کو برقرار رکھنے اور ترقی کی حمایت کرنے کے لئے بہت اہم ہیں۔

3. توانائی: اپنے بھینس کے وزن کو بڑھانے کے ل you ، آپ کو اضافی توانائی کے ذرائع جیسے مکئی ، گڑ ، یا شوگر چوقبصور کا گودا فراہم کرنے کی ضرورت پڑسکتی ہے۔ یہ آپ کے بھینس کو حاصل کرنے میں مدد کرسکتے ہیں


In [ ]:
!pip install gradio

import gradio as gr

def chat_interface(user_input, history=[]):
    reply = multilingual_chat(user_input)
    history.append((user_input, reply))
    return history, history

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(),
    textbox=gr.Textbox(placeholder="Ask me anything..."),
    title="Vet Chatbot"
).launch()


In [36]:
text = "### Instruction:\nWhat is the treatment for foot and mouth?\n\n### Response:"
tokens = tokenizer(text, return_tensors="pt")
print("input_ids:", tokens["input_ids"][0][:20])
print("decoded:", tokenizer.decode(tokens["input_ids"][0]))

input_ids: tensor([    1,   774,  3133,  3112, 28747,    13,  3195,   349,   272,  5827,
          354,  3331,   304,  5108, 28804,    13,    13, 27332, 12107, 28747])
decoded: <s> ### Instruction:
What is the treatment for foot and mouth?

### Response:


In [ ]:
text = "### Instruction:\nWhat is the treatment for foot and mouth?\n\n### Response: Antibiotics, rest, and isolation."
tokens = tokenizer(text, return_tensors="pt")
print("input_ids:", tokens["input_ids"][0][:30])
print("decoded:", tokenizer.decode(tokens["input_ids"][0]))

In [ ]:
trainer.train()
# tarined for some steps, then stopped it , training loss reduces

In [ ]:
# checkign the model
prompt = "### Instruction:\nAnimal: cow, Age: 3, Temperature: 103.1°F, Symptoms: depression, painless lumps, loss of appetite\n\n### Response:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# Saved model and tokenizer
model.save_pretrained("./finetuned-openchat-lora")
tokenizer.save_pretrained("./finetuned-openchat-lora")


In [ ]:
trainer.save_state()


In [ ]:
# resume with
# trainer.train(resume_from_checkpoint=True)